# 04 - AI Orchestration with Azure AI Search
**(Langchain / Python version)**

In this lab, we will do a deeper dive into using Azure AI Search as a vector store, the different search methods it supports and how you can use it as part of the Retrieval Augmented Generation (RAG) pattern for working with large language models.

## Create an Azure AI Search Vector Store in Azure

First, we will create an Azure AI Search service in Azure. The following are command line instructions and require the Azure CLI to be installed.

**NOTE:** Before running the commands, replace the **`<INITIALS>`** with your own initials or some random characters, as we need to provide a unique name for the Azure AI Search service.

In [6]:
RESOURCE_GROUP="azure-ai-search-rg"
LOCATION="westeurope"
NAME="ai-vectorstore-chw"
!az group create --name $RESOURCE_GROUP --location $LOCATION --subscription "Sandbox 1"
!az search service create -g $RESOURCE_GROUP -n $NAME -l $LOCATION --subscription "Sandbox 1" --sku Basic --partition-count 1 --replica-count 1

{
  "id": "/subscriptions/5de6ce13-eb75-4963-80be-88befe8b9845/resourceGroups/azure-ai-search-rg",
  "location": "westeurope",
  "managedBy": null,
  "name": "azure-ai-search-rg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
{- Finished ..
  "authOptions": {
    "apiKeyOnly": {}
  },
  "disableLocalAuth": false,
  "encryptionWithCmk": {
    "encryptionComplianceStatus": "Compliant",
    "enforcement": "Unspecified"
  },
  "hostingMode": "default",
  "id": "/subscriptions/5de6ce13-eb75-4963-80be-88befe8b9845/resourceGroups/azure-ai-search-rg/providers/Microsoft.Search/searchServices/ai-vectorstore-chw",
  "location": "West Europe",
  "name": "ai-vectorstore-chw",
  "networkRuleSet": {
    "ipRules": []
  },
  "partitionCount": 1,
  "privateEndpointConnections": [],
  "provisioningState": "succeeded",
  "publicNetworkAccess": "Enabled",
  "replicaCount": 1,
  "resourceGroup": "azure-ai-search-rg",
  "semantic

Next, we need to find and update the following values in the `.env` file with the Azure AI Search **name**, **endpoint** and **admin key** values, which you can get from the Azure portal. You also need to provide an **index name** value. The index will be created during this lab, so you can use any name you like.

```
AZURE_AI_SEARCH_SERVICE_NAME = "<YOUR AZURE AI SEARCH SERVICE NAME - e.g. ai-vectorstore-xyz>"
AZURE_AI_SEARCH_ENDPOINT = "<YOUR AZURE AI SEARCH ENDPOINT URL - e.g. https://ai-vectorstore-xyz.search.windows.net"
AZURE_AI_SEARCH_INDEX_NAME = "<YOUR AZURE AI SEARCH INDEX NAME - e.g. ai-search-index>"
AZURE_AI_SEARCH_API_KEY = "<YOUR AZURE AI SEARCH ADMIN API KEY - e.g. get this value from the Azure portal>"
```

## Load environment variable values
As with previous labs, we'll use the values from the `.env` file in the root of this repository.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("This lab exercise will use the following values:")
    print("Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
    print("Azure AI Search: " + os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"))
else: 
    print("No file .env found")

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_ai_search_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")

This lab exercise will use the following values:
Azure OpenAI Endpoint: https://openai-sweden-testing.openai.azure.com/
Azure AI Search: ai-vectorstore-chw


First, we will load the data from the movies.csv file and then extract a subset to load into the Azure AI Search index. We do this to help avoid the Azure OpenAI embedding limits and long loading times when inserting data into the index. We use a Langchain document loader to do this.

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='./movies.csv', source_column='original_title', encoding='utf-8', csv_args={'delimiter':',', 'fieldnames': ['id', 'original_language', 'original_title', 'popularity', 'release_date', 'vote_average', 'vote_count', 'genre', 'overview', 'revenue', 'runtime', 'tagline']})
data = loader.load()

# Rather than load all 500 movies into Azure AI search, we will use a
# smaller subset of movie data to make things quicker. The more movies you load,
# the more time it will take for embeddings to be generated.

data = data[1:51]
print('Loaded %s movies.' % len(data))

Loaded 50 movies.


During this lab, we will need to work with embeddings. We use embeddings to create a vector representation of a piece of text. We will need to create embeddings for the documents we want to store in our Azure AI Search index and also for the queries we want to use to search the index. We will create an Azure OpenAI client to do this.

In [3]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
)

## Create an Azure AI Search index and load movie data

Next, we'll step through the process of configuring an Azure AI Search index to store our movie data and then loading the data into the index. 

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    SemanticPrioritizedFields,
    SemanticSearch,
    SemanticField,
    SemanticConfiguration,
    SimpleField,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex
)
from azure.search.documents.models import (
    VectorizedQuery
)

When configuring an Azure AI Search index, we need to specify the fields we want to store in the index and the data types for each field. These match the fields in the movie data, containing values such as the movie title, genre, year of release and so on.

To use Azure AI Search as a vector store, we will also need to define a field to hold the vector representaion of the movie data. We indicate to Azure AI Search that this field will contain vector data by providing details of the vector dimensions and a profile. We'll also define the vector search configuration and profile with default values.

**NOTE:** It is possible just to use Azure AI Search as a vector store only, in which case we probably wouldn't need to define all of the index fields below. However, in this lab, we're also going to demonstrate Hybrid Search, a feature which makes use of both traditional keyword based search in combination with vector search.

In [5]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="overview", type=SearchFieldDataType.String),
    SearchableField(name="genre", type=SearchFieldDataType.String),
    SearchableField(name="tagline", type=SearchFieldDataType.String),
    SearchableField(name="release_date", type=SearchFieldDataType.DateTimeOffset, sortable=True),
    SearchableField(name="popularity", type=SearchFieldDataType.Double, sortable=True),
    SearchableField(name="vote_average", type=SearchFieldDataType.Double, sortable=True),
    SearchableField(name="vote_count", type=SearchFieldDataType.Int32, sortable=True),
    SearchableField(name="runtime", type=SearchFieldDataType.Int32, sortable=True),
    SearchableField(name="revenue", type=SearchFieldDataType.Int64, sortable=True),
    SearchableField(name="original_language", type=SearchFieldDataType.String),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536, vector_search_profile_name="movies-vector-profile"),
]

vector_search = VectorSearch(
    profiles=[VectorSearchProfile(name="movies-vector-profile", algorithm_configuration_name="movies-vector-config")],
    algorithms=[HnswAlgorithmConfiguration(name="movies-vector-config")],
)

We're going to be using Semantic Ranking, a feature of Azure AI Search that improves search results by using language understanding to rerank the search results. We provide a Semantic Search Configuration to help the ranking model understand the movie data, by telling it which fields contain the movie title, which fields contain keywords and which fields contain general free text content.

In [6]:
semantic_config = SemanticConfiguration(
    name="movies-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="genre")],
        content_fields=[SemanticField(field_name="title"),
                        SemanticField(field_name="overview"),
                        SemanticField(field_name="tagline"),
                        SemanticField(field_name="genre"),
                        SemanticField(field_name="release_date"),
                        SemanticField(field_name="popularity"),
                        SemanticField(field_name="vote_average"),
                        SemanticField(field_name="vote_count"),
                        SemanticField(field_name="runtime"),
                        SemanticField(field_name="revenue"),
                        SemanticField(field_name="original_language")],
    )
)

semantic_search = SemanticSearch(configurations=[semantic_config])

Finally, we'll go ahead and create the index by creating an instance of the `SearchIndex` class and adding the keyword and vectors fields and the semantic search profile.

In [7]:
# Create the search index with the desired vector search and semantic configurations
index = SearchIndex(
    name=azure_ai_search_index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search
)

index_client = SearchIndexClient(
    azure_ai_search_endpoint,
    AzureKeyCredential(azure_ai_search_api_key)
)

result = index_client.create_or_update_index(index)

print(f'Index {result.name} created.')

Index ai-search-index created.


The index is now ready, so next we need to prepare the movie data to load into the index.

**NOTE**: During this phase, we send the data for each movie to an Azure OpenAI embeddings model to create the vector data. This may take some time due to rate limiting in the API.

In [8]:
# Loop through all of the movies and create a new item for each one.

items = []
for movie in data:
    content = movie.page_content
    fields = movie.page_content.split('\n')
    movieId = (fields[0].split(': ')[1])[:-2]
    movieTitle = (fields[2].split(': ')[1])
    movieOverview = (fields[8].split(': ')[1])
    movieGenre = (fields[7].split(': ')[1])[1:-1]
    movieTagline = (fields[11].split(': ')[1])
    movieReleaseDate = (fields[4].split(': ')[1])
    moviePopularity = (fields[3].split(': ')[1])
    movieVoteAverage = (fields[5].split(': ')[1])
    movieVoteCount = (fields[6].split(': ')[1])
    movieRuntime = (fields[10].split(': ')[1])
    movieRevenue = (fields[9].split(': ')[1])
    movieOriginalLanguage = (fields[1].split(': ')[1])

    items.append(dict([
        ("id", movieId), 
        ("title", movieTitle),
        ("overview", movieOverview),
        ("genre", movieGenre),
        ("tagline", movieTagline),
        ("release_date", movieReleaseDate),
        ("popularity", moviePopularity),
        ("vote_average", movieVoteAverage),
        ("vote_count", movieVoteCount),
        ("runtime", movieRuntime),
        ("revenue", movieRevenue),
        ("original_language", movieOriginalLanguage),
        ("vector", azure_openai_embeddings.embed_query(content))
    ]))

    print(f"Movie {movieTitle} added.")

print(f"New items structure with embeddings created for {len(items)} movies.")

Movie Hidden Figures added.
Movie Gridlocked added.
Movie Joker added.
Movie The Sand added.
Movie America added.
Movie 僕のヒーローアカデミア THE MOVIE ～2人の英雄～ added.
Movie Under Siege 2 added.
Movie The Enforcer added.
Movie Ruby Sparks added.
Movie They Came Together added.
Movie The Handmaid's Tale added.
Movie Броненосец Потёмкин added.
Movie Enemy of the State added.
Movie Pistol Whipped added.
Movie 맛있는 비행 added.
Movie Wheelman added.
Movie Raising Arizona added.
Movie Rampage added.
Movie Evolution added.
Movie Man on Wire added.
Movie Work It added.
Movie Step Sisters added.
Movie Pirates of the Caribbean added.
Movie I Don't Know How She Does It added.
Movie The Wrestler added.
Movie The Sisterhood of the Traveling Pants added.
Movie Charlie Wilson's War added.
Movie El Infierno added.
Movie Una última y nos vamos added.
Movie Immortals added.
Movie Pandorum added.
Movie Women added.
Movie Hotel Transylvania 3 added.
Movie Alex Strangelove added.
Movie The Painted Veil added.
Movie John

We can write out the contents of one of the documents to see what it looks like. You can see that it contains the movie data at the top and then a long array containing the vector data.

In [9]:
print(items[0])

{'id': '381284', 'title': 'Hidden Figures', 'overview': 'The untold story of Katherine G. Johnson, Dorothy Vaughan and Mary Jackson – brilliant African-American women working at NASA and serving as the brains behind one of the greatest operations in history – the launch of astronaut John Glenn into orbit. The visionary trio crossed all gender and race lines to inspire generations to dream big.', 'genre': "'Drama', 'History'", 'tagline': "Meet the women you don't know, behind the mission you do.", 'release_date': '2016-12-10', 'popularity': '49.802', 'vote_average': '8.1', 'vote_count': '7310.0', 'runtime': '127.0', 'revenue': '230698791.0', 'original_language': 'en', 'vector': [-0.007291263298045165, -0.022922893156763043, -0.020431273606514578, -0.028535592724720317, 0.01314001123979203, -0.007586323532451499, -0.003109605890241679, -0.021886904779222414, -0.03973476395310626, -0.03524984950771712, 0.01977558523152563, 0.011127044948343589, -0.01740199033383327, -0.03469906903755212, 

Now we have the movie data stored in the correct format, so let's load it into the Azure AI Search index we created earlier.

In [10]:
from azure.search.documents import SearchClient

search_client = SearchClient(
    azure_ai_search_endpoint,
    azure_ai_search_index_name,
    AzureKeyCredential(azure_ai_search_api_key)
)

result = search_client.upload_documents(items)

print(f"Successfully loaded {len(data)} movies into Azure AI Search index.")

Successfully loaded 50 movies into Azure AI Search index.


## Vector store searching using Azure AI Search

We've loaded the movies into Azure AI Search, so now let's experiment with some of the different types of searches you can perform.

First we'll just perform a simple keyword search.

In [11]:
query = "hero"

results = list(search_client.search(
    search_text=query,
    query_type="simple",
    include_total_count=True,
    top=5
))

for result in results:
    print("Movie: {}".format(result["title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")

Movie: Max
Genre: 'Adventure', 'Drama'
----------
Movie: Immortals
Genre: 'Fantasy', 'Action', 'Drama'
----------
Movie: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Genre: 'Animation', 'Action', 'Adventure', 'Fantasy'
----------


We get some results, but they're not necessarily movies about heroes. It could be that there is some text in the index for these results that relates to the word "hero". For example, the description might mention "heroic deeds" or something similar.

Let's now try the same again, but this time we'll ask a question instead of just searching for a keyword.

In [12]:
query = "What are the best movies about superheroes?"

results = list(search_client.search(
    search_text=query,
    query_type="simple",
    include_total_count=True,
    top=5
))

for result in results:
    print("Movie: {}".format(result["title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")

Movie: The Miseducation of Cameron Post
Genre: 'Drama'
----------
Movie: The Sisterhood of the Traveling Pants
Genre: 'Drama', 'Comedy'
----------
Movie: Perfume
Genre: 'Crime', 'Fantasy', 'Drama'
----------
Movie: Max
Genre: 'Adventure', 'Drama'
----------
Movie: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Genre: 'Animation', 'Action', 'Adventure', 'Fantasy'
----------


As before, you will likely get mixed results. Some of the movies returned could be about heroes, but others may not be. This is because the search is still based on keywords.

Next, let's try a vector search.

In [14]:
query = "What are the best movies about superheroes?"

vector = VectorizedQuery(vector=azure_openai_embeddings.embed_query(query), k_nearest_neighbors=5, fields="vector")

# Note the `None` value for the `search_text` parameter. This is because we're not sending the query text to Azure AI Search. We're sending the embedded version of the query text instead via the `vector_queries` parameter.

results = list(search_client.search(
    search_text=None,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    vector_queries=[vector],
    select=["title", "genre"],
    top=5
))

for result in results:
    print("Movie: {}".format(result["title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")

Movie: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Genre: 'Animation', 'Action', 'Adventure', 'Fantasy'
----------
Movie: Immortals
Genre: 'Fantasy', 'Action', 'Drama'
----------
Movie: America
Genre: 'Action', 'Comedy', 'History', 'Animation', 'Fantasy'
----------
Movie: Joker
Genre: 'Crime', 'Thriller', 'Drama'
----------
Movie: Legends of Oz
Genre: 'Animation', 'Family', 'Fantasy'
----------


It's likely that the raw vector search didn't return exactly what you were expecting. You were probably expecting a list of superhero movies, but now we're getting a list of movies that are **similar** to the vector we provided. Some of these may be hero movies, but others may not be. The vector search is returning the nearest neighbours to the vector we provided, so it's possible that at least one of the results is a superhero movie, and the others are similar to that movie in some way.

So, both the keyword search and the vector search have their limitations. The keyword search is limited to the keywords in the index, so it's possible that we might miss some movies that are about heroes. The vector search is limited to returning the nearest neighbours to the vector we provide, so it's possible that we might get some movies that are not about heroes.

## Hybrid search using Azure AI Search

To overcome the limitations of both keyword search and vector search, we can use a combination of both. This is known as Hybrid Search. Let's run the same query again, but this time we'll use Hybrid Search.

The only significant difference is that this time we will submit both the original query text and the embedding vector to Azure AI Search. Azure AI Search will then use both the query text and the vector to perform the search and combine the results.

In [15]:
query = "What are the best movies about superheroes?"

vector = VectorizedQuery(vector=azure_openai_embeddings.embed_query(query), k_nearest_neighbors=5, fields="vector")

# Note the `None` value for the `search_text` parameter. This is because we're not sending the query text to Azure AI Search. We're sending the embedded version of the query text instead via the `vector_queries` parameter.

results = list(search_client.search(
    search_text=query,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    vector_queries=[vector],
    select=["title", "genre"],
    top=5
))

for result in results:
    print("Movie: {}".format(result["title"]))
    print("Genre: {}".format(result["genre"]))
    print("Score: {}".format(result["@search.score"]))
    print("Reranked score: {}".format(result["@search.reranker_score"]))
    print("----------")

Movie: Immortals
Genre: 'Fantasy', 'Action', 'Drama'
Score: 0.029213953763246536
Reranked score: 2.4780943393707275
----------
Movie: 僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Genre: 'Animation', 'Action', 'Adventure', 'Fantasy'
Score: 0.03229166567325592
Reranked score: 2.4727327823638916
----------
Movie: Joker
Genre: 'Crime', 'Thriller', 'Drama'
Score: 0.02736726962029934
Reranked score: 2.3302695751190186
----------
Movie: The Enforcer
Genre: 'Action', 'Crime', 'Thriller'
Score: 0.013888888992369175
Reranked score: 2.2569315433502197
----------
Movie: Pandorum
Genre: 'Action', 'Horror', 'Mystery', 'Science Fiction', 'Thriller'
Score: 0.012987012974917889
Reranked score: 2.0923521518707275
----------


Hopefully, you'll now see a much better set of results. Performing a hybrid search has allowed us to combine the benefits of both keyword search and vector search. But also, Azure AI Search performs a further step when using hybrid search. It makes use of a Semantic Ranker to further improve the search results. The Semantic Ranker uses a language understanding model to understand the query text and the documents in the index and then uses this information to rerank the search results. So, after performing the keyword and vector search, Azure AI Search will then use the Semantic Ranker to re-order the search results based on the context of the original query.

In the results above, you can see a `Reranked Score`. This is the score that has been calculated by the Semantic Ranker. The `Score` is the score calculated by the keyword and vector search. You'll note that the results are returned in the order determined by the reranked score.

## Bringing it All Together with Retrieval Augmented Generation (RAG) + Langchain (LC)

Now that we have our Vector Store setup and data loaded, we are now ready to implement the RAG pattern using AI Orchestration. At a high-level, the following steps are required:
1. Ask the question
2. Create Prompt Template with inputs
3. Get Embedding representation of inputted question
4. Use embedded version of the question to search Azure AI Search (ie. The Vector Store)
5. Inject the results of the search into the Prompt Template & Execute the Prompt to get the completion

In [16]:
# Implement RAG using Langchain (LC)

from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain


azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

azure_openai = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

# Ask the question
query = "What are the best movies about superheroes?"

# Create a prompt template with variables, note the curly braces
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["original_question","search_results"],
    template="""
    Question: {original_question}

    Do not use any other data.
    Only use the movie data below when responding.
    Provide detailed information about the synopsis of the movie.
    {search_results}
    """,
)

# Search Vector Store
search_client = SearchClient(
    azure_ai_search_endpoint,
    azure_ai_search_index_name,
    AzureKeyCredential(azure_ai_search_api_key)
)

vector = VectorizedQuery(vector=azure_openai_embeddings.embed_query(query), k_nearest_neighbors=5, fields="vector")

results = list(search_client.search(
    search_text=query,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    include_total_count=True,
    vector_queries=[vector],
    select=["title","genre","overview","tagline","release_date","popularity","vote_average","vote_count","runtime","revenue","original_language"],
    top=5
))

# Build the Prompt and Execute against the Azure OpenAI to get the completion
chain = LLMChain(llm=azure_openai, prompt=prompt, verbose=False)
response = chain.invoke({"original_question": query, "search_results": results})
print(response['text'])

Based on the provided movie data, the best movie about superheroes according to the highest vote average is "僕のヒーローアカデミア THE MOVIE ～2人の英雄～" (My Hero Academia: Two Heroes) with a vote average of 8.0. 

My Hero Academia: Two Heroes is an animated action-adventure movie set in the universe of the popular "My Hero Academia" series. The movie's synopsis is as follows:

The film centers around the protagonist, All Might, and his protege, Deku (Izuku Midoriya), as they receive an invitation to visit a floating and mobile manmade city called 'I-Island'. This island is a hub for researching quirks, which are the superpowers that individuals in this universe may possess, as well as hero supplemental items. A special event named the 'I-Expo' convention is currently being held on the island to showcase the latest hero equipment and research.

During their time on the island, despite stringent security measures, a villain breaches the system. The only ones capable of thwarting this new threat are t

## Next Section

📣 [Deploy AI](../../04-deploy-ai/README.md)